# "iso 1861 UTC" "lon" "lat"  transfer to local time

## Method 1. Bing Map API

use bing map API https://docs.microsoft.com/en-us/bingmaps/rest-services/timezone/convert-local-time-zone

In [1]:
import datetime
import json
import os
import requests
import pandas as pd

In [2]:
# Using Bing Maps API
BING_MAPS_KEY ="AkxA0kVoCBBJRjjHQW5T63uvunkM5Ln5QLkl4u_qo5U5GRPDH32qYBp1_yWHX52N"
print(BING_MAPS_KEY )

AkxA0kVoCBBJRjjHQW5T63uvunkM5Ln5QLkl4u_qo5U5GRPDH32qYBp1_yWHX52N


In [3]:
#working
def get_time_zone(lon,lat,isotime_utc):
    try: 
        r = requests.get(
        f"https://dev.virtualearth.net/REST/v1/TimeZone/{lat},{lon}?datetime={isotime_utc}&key={BING_MAPS_KEY}")
    
        return r.json()['resourceSets'][0]['resources'][0]['timeZone']['convertedTime']['localTime']
    except:
        return "NA"

In [4]:
print(get_time_zone(-78.853563,42.8962295,"2019-03-31T23:57:12.000Z"))

2019-03-31T19:57:12


In [5]:
#csv add local_time column
import pandas as pd
df= pd.read_csv(r"E:\OneDrive - University of Georgia\Project\Data\tweet_data_2_food\4.1 LocalTime\fastFoodResturant_bboxStateLevel_localTime2019_1.csv",low_memory=False)

In [ ]:
pd.options.mode.chained_assignment = None
i=0
#609937
for row in df.head(75000).iterrows():
    if  str(df['local_time'][i])=="nan":
        df['local_time'][i]= get_time_zone(df["lon"][i],df["lat"][i],df["created_at"][i])  
        print(i,df["created_at"][i],df['local_time'][i])
    #print(i)
    i+=1

In [ ]:
df.to_csv(r"E:\OneDrive - University of Georgia\Project\Data\tweet_data_2_food\4.1 LocalTime\fastFoodResturant_bboxStateLevel_localTime2019_1.csv",encoding="utf_8_sig",index=False)

## Method2. Use datetime and pytz library

In [ ]:
pd.options.mode.chained_assignment = None
i=0

for row in df.head(90000).iterrows():
    if  str(df['local_time'][i])=="nan":
        df['local_time'][i]= get_time_zone(df["lon"][i],df["lat"][i],df["created_at"][i])  
        print(i,df["created_at"][i],df['local_time'][i])
    #print(i)
    i+=1

In [ ]:
from tzwhere import tzwhere
from pytz import timezone
import datetime
import pytz
import time
from dateutil import parser

In [ ]:
#iso to timestamp
import datetime,pytz

def iso2timestamp(datestring, format='%Y-%m-%dT%H:%M:%S.%fZ',timespec='seconds'):
    """
    ISO8601时间转换为时间戳

    :param datestring:iso时间字符串 2019-03-25T16:00:00.000Z，2019-03-25T16:00:00.000111Z
    :param format:%Y-%m-%dT%H:%M:%S.%fZ；其中%f 表示毫秒或者微秒
    :param timespec:返回时间戳最小单位 seconds 秒，milliseconds 毫秒,microseconds 微秒
    :return:时间戳 默认单位秒
    """
    tz = pytz.timezone('Asia/Shanghai')
    utc_time = datetime.datetime.strptime(datestring, format)  # 将字符串读取为 时间 class datetime.datetime

    time = utc_time.replace(tzinfo=pytz.utc).astimezone(tz)

    times = {
        'seconds': int(time.timestamp()),
        'milliseconds': round(time.timestamp() * 1000),
        'microseconds': round(time.timestamp() * 1000 * 1000),
    }
    return times[timespec]

In [ ]:
def get_local_time(lon,lat,isotime_utc):
    
    try: 
        #iso to timestamp
        timestamp = iso2timestamp(isotime_utc)
    
        #timestamp, lat, lon to local_time
        tz = tzwhere.tzwhere(forceTZ=True)
        time_zone = tz.tzNameAt(lat,lon, forceTZ=True)  #纬度，经度。 得到对应的时区
        #print(timestamp)
        local_time = datetime.datetime.fromtimestamp(timestamp,pytz.timezone(time_zone)).strftime('%Y-%m-%dT%H:%M:%S')
    
        return str(local_time)
    
    except:
        return "NA"

In [ ]:
print(get_local_time(-78.853563,42.8962295,"2019-03-31T23:57:12.000Z"))

In [ ]:
#csv add local_time column
import pandas as pd
df= pd.read_csv(r"Estate_level.csv",low_memory=False) 

In [ ]:
#TRY THIS
#Recalculate the NA timezone
pd.options.mode.chained_assignment = None
i=0

for row in df.iterrows():
    if  str(df['local_time'][i])=="nan":
        df['local_time'][i]= get_local_time(df["lon"][i],df["lat"][i],df["created_at"][i])  
        print(i)
        print(df['local_time'][i])
   
    i+=1

In [ ]:
df.to_csv(r"E:\Project\Data\tweet_data\3.1 FilterBBox4Level_4csv\2.csv",encoding="utf_8_sig",index=False)